# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Linz  ->  H. Linz  |  ['H. Linz']
S. Li  ->  S. Li  |  ['S. Li']
Arxiv has 51 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2306.07394


extracting tarball to tmp_2306.07394...

 done.


H. Linz  ->  H. Linz  |  ['H. Linz']


Found 86 bibliographic references in tmp_2306.07394/OH_Outflow_G34_VLA.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.07394-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.07394) | **Excited Hydroxyl Outflow in the High-Mass Star-Forming Region  G34.26+0.15**  |
|| W. S. Tan, et al. -- incl., <mark>H. Linz</mark> |
|*Appeared on*| *2023-06-14*|
|*Comments*| *19 pages, 6 figures. Accepted for publication in The Astrophysical Journal*|
|**Abstract**| G34.26+0.15 is a region of high-mass star formation that contains a broad range of young stellar objects in different stages of evolution, including a hot molecular core, hyper-compact HII regions and a prototypical cometary ultra-compact HII region. Previous high-sensitivity single dish observations by our group resulted in the detection of broad 6035 MHz OH absorption in this region; the line showed a significant blue-shifted asymmetry indicative of molecular gas expansion. We present high-sensitivity Karl G. Jansky Very Large Array (VLA) observations of the 6035 MHz OH line conducted to image the absorption and investigate its origin with respect to the different star formation sites in the region. In addition, we report detection of 6030 MHz OH absorption with the VLA and further observations of 4.7 GHz and 6.0 GHz OH lines obtained with the Arecibo Telescope. The 6030 MHz OH line shows a very similar absorption profile as the 6035 MHz OH line. We found that the 6035 MHz OH line absorption region is spatially unresolved at $\sim 2$" scales, and it is coincident with one of the bright ionized cores of the cometary HII region that shows broad radio recombination line emission. We discuss a scenario where the OH absorption is tracing the remnants of a pole-on molecular outflow that is being ionized inside-out by the ultra-compact HII region. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2306.07394.md
    + _build/html/tmp_2306.07394/./merged_image_EA_v1.png
    + _build/html/tmp_2306.07394/./Arecibo_spectra.png
    + _build/html/tmp_2306.07394/./oh_simulated_data.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\kms }{ km s^{-1}}$</div>



<div id="title">

# Excited Hydroxyl Outflow in the High-Mass Star-Forming Region G34.26+0.15

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2306.07394-b31b1b.svg)](https://arxiv.org/abs/2306.07394)<mark>Appeared on: 2023-06-14</mark> -  _19 pages, 6 figures. Accepted for publication in The Astrophysical Journal_

</div>
<div id="authors">

W. S. Tan, et al. -- incl., <mark>H. Linz</mark>

</div>
<div id="abstract">

**Abstract:** G34.26+0.15 is a region of high-mass star formation that contains a broad range of young stellar objects in different stages of evolution, including a hot molecular core, hyper-compact H ${\small II}$ regions and a prototypical cometary ultra-compact H ${\small II}$ region. Previous high-sensitivity single dish observations by our group resulted in the detection of broad 6035 $ $ MHz OH absorption in this region; the line showed a significant blue-shifted asymmetry indicative of molecular gas expansion. We present high-sensitivity Karl G. Jansky Very Large Array (VLA) observations of the 6035 $ $ MHz OH line conducted to image the absorption and investigate its origin with respect to the different star formation sites in the region. In addition, we report detection of 6030 $ $ MHz OH absorption with the VLA and further observations of 4.7 $ $ GHz and 6.0 $ $ GHz OH lines obtained with the Arecibo Telescope. The 6030 $ $ MHz OH line shows a very similar absorption profile as the 6035 $ $ MHz OH line. We found that the 6035 $ $ MHz OH line absorption region is spatially unresolved at $\sim 2$ $\arcsec$ scales, and it is coincident with one of the bright ionized cores of the cometary H ${\small II}$ region that shows broad radio recombination line emission. We discuss a scenario where the OH absorption is tracing the remnants of a pole-on molecular outflow that is being ionized inside-out by the ultra-compact H ${\small II}$ region.

</div>

<div id="div_fig1">

<img src="tmp_2306.07394/./merged_image_EA_v1.png" alt="Fig3" width="100%"/>

**Figure 3. -** \small Spectra of the 6.0$ $GHz OH transitions obtained with the Arecibo Telescope (red), VLA in D-configuration (blue) and B-configuration (black) are shown in the left panels; the spectra are sorted by frequency: 6016$ $MHz, 6030$ $MHz, 6035$ $MHz, and 6049$ $MHz from top to bottom. Radio continuum obtained from the line-free channels of the respective spectral windows is shown in the middle (D-configuration) and right panels (B-configuration). In the middle panels, we show with a red circle the $\sim$44$\arcsec$ Arecibo beam, centered at the pointing position; the blue contours in the 6030 and 6035$ $MHz panels show the velocity integrated intensity (moment-0) images of the absorption from the VLA D-configuration cubes (6030$ $MHz OH absorption contour levels: $-$10, $-$90, $-$170, $-$250 $\times $3.08$ $mJy$ $km$ $s$^{-1}$; 6035$ $MHz OH absorption contour levels: $-$70, $-$180, $-$290, $-$400 $\times $2.65$ $mJy$ $km$ $s$^{-1}$). No detection was obtained in the D-configuration observations of the 6016$ $MHz (middle, upper panel) nor the 6049$ $MHz (middle, lower panel) OH transitions; for these transitions, we show with a blue circle the region used to obtain the blue spectrum shown in the respective left panels. The right panels are equivalent to the middle panels, but for the B-configuration observations; black circles are the regions integrated to obtain the black spectra shown in the right panels, and the moment-0 image of the 6035$ $MHz OH absorption is shown in black contours ($-$60, $-$140, $-$220, $-$300 $\times $2.19$ $mJy$ $km$ $s$^{-1}$). The spectral window of the 6030$ $MHz VLA-B observations was corrupted. (*fig:AO-VLA_Spectra_and_Images*)

</div>
<div id="div_fig2">

<img src="tmp_2306.07394/./Arecibo_spectra.png" alt="Fig2" width="100%"/>

**Figure 2. -** OH spectra obtained with the Arecibo Telescope. The left panels show the 4.7$ $GHz OH transitions and the right panels show the 6.0$ $GHz lines (the end of the bandpass can be seen at $-$40$\kms$ in the 6.0$ $GHz spectra). We report detection of emission in the 4750$ $MHz and 4765$ $MHz transitions (the 4660$ $MHz may also show a narrow and weak emission line, however, only one channel is above 3$\sigma$). Maser lines are detected in the 6035$ $MHz transition (see  ([Al-Marzouk, Araya and Hofner 2012]()) ), and absorption is found in the 6035$ $MHz, 6030$ $MHz and 6016$ $MHz transitions. The 6016$ $MHz line is weak; the spectra had to be smoothed to a channel width of 7.6$\kms$ to detect the absorption. We report no detection of the 6049$ $MHz transition (no clear evidence of absorption is detected when the spectrum is smoothed). (*fig:Arecibo_spectra*)

</div>
<div id="div_fig3">

<img src="tmp_2306.07394/./oh_simulated_data.png" alt="Fig6" width="100%"/>

**Figure 6. -** Predicted integrated flux density of the \textcolor{blue}{6016$ $MHz}(blue solid line),
\textcolor{DrakGreen}{6030$ $MHz}(green solid line), \textcolor{red}{6035$ $MHz}(red solid line), and
\textcolor{LimeBlue}{6049$ $MHz}(dark-cyan dashed line) OH transitions as a function of column density
computed with {\tt MOLPOP-CEP}(model assumes T$_{k} = 150  $K, $n_{H_2} = 10^{5} $cm$^{-3}$,
$ \chi_{OH} = {5\times10^{-6}}$, FWHM = 33$ $km$ $s$^{-1}$, FWHM$_{6016MHz}$ = 17$ $km$ $s$^{-1}$). The horizontal bands show selected integrated flux density values (including uncertainties) as listed in
Table \ref{tab:OH_Line_Parameters}. The vertical dashed line shows a column density for which the model
matches the observations.
 (*fig:oh_Line_parameters*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2306.07394"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

220  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
